In [8]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent, AgentState
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.types import Command
from langchain.messages import HumanMessage
from pprint import pprint


In [9]:
load_dotenv()

True

In [10]:
@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""

    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""

    # fake email sending
    return "Email sent"

In [11]:
class EmailState(AgentState):
    email: str

In [12]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")

In [13]:
agent = create_agent(
    model=model,
    tools = [read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True
            },
            description_prefix="Tool execution requires approval"
        )
    ]
)

In [14]:
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now it tne same thread.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [15]:
pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now it tne same thread.', additional_kwargs={}, response_metadata={}, id='d47ede7c-ee13-4169-969f-6fec0cff934b'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'read_email', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'8a36ceac-0dee-49ea-bd9a-8617acd73a17': 'CoAFAXLI2nza31q0kaBh7RvPsPv5eMY82/vMw5tu4UgCS4bM8wkYYhBUhwgedLeV7POT7VdNRydXsnB4Bcxuv0eDBv8e5lve6n9ZuS+hzjvNPiYURmB0HB4qTgiXmK4Cf48rS/m6pj4VzpFj7qbevn2FCG0yd1SMw9YYiUyIYDtk2ZS3QaVT6H6XigOTitcOdnkNd7qn/b6bvldBrqaHfmebHw0gCb5XzSVZtfvIsDAg/lWAWm+3dEwuQBzl1R0HmDubBVBmdsPlCSUhuzci4zHzQdf+3No8k3IF3L43R4UzqSchhiKmhomKW7YM+k9uAbrWnhFRjT8Lz1yzskLOEwT/D93ImY+ADeMU5eAMuQZYl5FdA0+5k+miWT6m5NfoqVn9V2Vjod+FzsQMuabXxYifbZ2Z/UASihFs6QZnBX+tsqgyRzBk8kiTGWjb9uxne7Bz6tF08tSQ5+wW

# Approve

In [17]:
response = agent.invoke(
    Command(
        resume={"decisions": [{"type": "approve"}]},
    ),
    config=config # Same thread id to resume the paused conversation
)

In [18]:
pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now it tne same thread.', additional_kwargs={}, response_metadata={}, id='d47ede7c-ee13-4169-969f-6fec0cff934b'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'read_email', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'8a36ceac-0dee-49ea-bd9a-8617acd73a17': 'CoAFAXLI2nza31q0kaBh7RvPsPv5eMY82/vMw5tu4UgCS4bM8wkYYhBUhwgedLeV7POT7VdNRydXsnB4Bcxuv0eDBv8e5lve6n9ZuS+hzjvNPiYURmB0HB4qTgiXmK4Cf48rS/m6pj4VzpFj7qbevn2FCG0yd1SMw9YYiUyIYDtk2ZS3QaVT6H6XigOTitcOdnkNd7qn/b6bvldBrqaHfmebHw0gCb5XzSVZtfvIsDAg/lWAWm+3dEwuQBzl1R0HmDubBVBmdsPlCSUhuzci4zHzQdf+3No8k3IF3L43R4UzqSchhiKmhomKW7YM+k9uAbrWnhFRjT8Lz1yzskLOEwT/D93ImY+ADeMU5eAMuQZYl5FdA0+5k+miWT6m5NfoqVn9V2Vjod+FzsQMuabXxYifbZ2Z/UASihFs6QZnBX+tsqgyRzBk8kiTGWjb9uxne7Bz6tF08tSQ5+wW

# Reject

In [ ]:
response = agent.invoke(
    Command(
        resume= {
            "decisions": [
                {
                    "type": "reject",
                    "message": "No please sing off - Your merciful leader, Seán"
                }
            ]
        }
    ),
    config
)

In [20]:
pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now it tne same thread.', additional_kwargs={}, response_metadata={}, id='d47ede7c-ee13-4169-969f-6fec0cff934b'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'read_email', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'8a36ceac-0dee-49ea-bd9a-8617acd73a17': 'CoAFAXLI2nza31q0kaBh7RvPsPv5eMY82/vMw5tu4UgCS4bM8wkYYhBUhwgedLeV7POT7VdNRydXsnB4Bcxuv0eDBv8e5lve6n9ZuS+hzjvNPiYURmB0HB4qTgiXmK4Cf48rS/m6pj4VzpFj7qbevn2FCG0yd1SMw9YYiUyIYDtk2ZS3QaVT6H6XigOTitcOdnkNd7qn/b6bvldBrqaHfmebHw0gCb5XzSVZtfvIsDAg/lWAWm+3dEwuQBzl1R0HmDubBVBmdsPlCSUhuzci4zHzQdf+3No8k3IF3L43R4UzqSchhiKmhomKW7YM+k9uAbrWnhFRjT8Lz1yzskLOEwT/D93ImY+ADeMU5eAMuQZYl5FdA0+5k+miWT6m5NfoqVn9V2Vjod+FzsQMuabXxYifbZ2Z/UASihFs6QZnBX+tsqgyRzBk8kiTGWjb9uxne7Bz6tF08tSQ5+wW

# Edit

In [21]:
response = agent.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type": "edit",
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ),
    config=config
)

In [22]:
pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now it tne same thread.', additional_kwargs={}, response_metadata={}, id='d47ede7c-ee13-4169-969f-6fec0cff934b'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'read_email', 'arguments': '{}'}, '__gemini_function_call_thought_signatures__': {'8a36ceac-0dee-49ea-bd9a-8617acd73a17': 'CoAFAXLI2nza31q0kaBh7RvPsPv5eMY82/vMw5tu4UgCS4bM8wkYYhBUhwgedLeV7POT7VdNRydXsnB4Bcxuv0eDBv8e5lve6n9ZuS+hzjvNPiYURmB0HB4qTgiXmK4Cf48rS/m6pj4VzpFj7qbevn2FCG0yd1SMw9YYiUyIYDtk2ZS3QaVT6H6XigOTitcOdnkNd7qn/b6bvldBrqaHfmebHw0gCb5XzSVZtfvIsDAg/lWAWm+3dEwuQBzl1R0HmDubBVBmdsPlCSUhuzci4zHzQdf+3No8k3IF3L43R4UzqSchhiKmhomKW7YM+k9uAbrWnhFRjT8Lz1yzskLOEwT/D93ImY+ADeMU5eAMuQZYl5FdA0+5k+miWT6m5NfoqVn9V2Vjod+FzsQMuabXxYifbZ2Z/UASihFs6QZnBX+tsqgyRzBk8kiTGWjb9uxne7Bz6tF08tSQ5+wW